## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Curve
https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [20]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum'

6-Month Average Daily Volume

In [21]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [22]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [23]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [24]:
df['dailyVolumeUSD'].astype(float).mean()

555447485.5768127

6-Month Average TVL

In [25]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/curve.csv")

In [27]:
lma

,Unnamed: 0,Date,Timestamp,Curve,Curve.1,Curve.2,Curve.3,Curve.4,Curve.5,Curve.6,...,Curve.67,Curve.68,Curve.69,Curve.70,Curve.71,Curve.72,Curve.73,Curve.74,Curve.75,Curve.76
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,optimism,avalanche,Total,harmony,moonbeam,ethereum,xdai,...,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum
2,NaN,NaN,NaN,TVL,TVL,TVL,TVL,TVL,TVL,TVL,...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WBTC,SBTC,HBTC,USDK,USDN,GUSD,HUSD,MUSD,RSV,TBTC
4,NaN,09/02/2020,1.581206e+09,NaN,NaN,1163733.163555646,NaN,NaN,1163733.163555646,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,NaN,14/07/2022,1.657757e+09,11683365.414221896,128940272.11470175,5112191464.164293,1527087.6653723603,5188420.818581765,4550925481.900369,71698933.39436468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
891,NaN,15/07/2022,1.657843e+09,11321648.328221604,127926200.70640086,5252916006.444021,1529924.8881579326,6709261.666706267,4697843120.0692625,71328344.12069276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892,NaN,16/07/2022,1.657930e+09,11503164.188707896,127717819.57878709,5319860006.023603,1531080.9717024018,7676767.764754455,4764679080.819227,71046771.70274982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
893,NaN,17/07/2022,1.658016e+09,11328092.419865165,128034825.70951968,5490801031.700195,1531830.6740342472,8194291.556497392,4931432286.945959,71136594.64468202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 8].astype(float).mean()

12866343024.997925

Largest Pool Concentration

In [43]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_3055/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [44]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x52ea46506b9cc5ef470c5bf89f17dc28bb35d85c,Curve.fi cDAI/cUSDC/USDT,4727483.62146723,325882589.7771241412015846575,1581353158,9456293
1,0x45f783cce6b7ff23b2ab2d70e416cdb7d6055f51,Curve.fi yDAI/yUSDC/yUSDT/yTUSD,9896684.778118787419320378,4164216137.306760717032287986,1581620865,9476468
2,0xa2b47e3d5c44877cca798226b7b8118f9bfb7a56,Curve.fi cDAI/cUSDC,71682037.07791614436485609179162948,35998107.56220225854876836720593083,1582652911,9554040
3,0x79a8c46dea5ada233abaffd40f3a0a2b1e5a4f27,Curve.fi yDAI/yUSDC/yUSDT/yBUSD,9046135.661778435912314576,1655243106.3415161114471576105,1582828615,9567295
4,0xa5407eae9ba41422680e2e00537571bcc53efbfd,Curve.fi DAI/USDC/USDT/sUSD,139237888.072652683478431597,16965620690.0295902165960410825,1587348844,9906598
...,...,...,...,...,...,...
334,0xb30da2376f63de30b42dc055c93fa474f31330a5,Curve.fi Factory USD Metapool: alUSDFRAXBP,17003.987376552850958636,3149.631549823209645115,1657597179,15125488
335,0xd3301b7caa76f932816a6fc7ef0b673238e217ad,Curve.fi Factory Crypto Pool: BENT/FRAX,0,0,1657678738,15131595
336,0xb2e2536821521174a168eda7be78a6c70ea6e5fa,Curve.fi Factory Crypto Pool: BENTCVX/FRAX,0,0,1657678861,15131603
337,0x316d2606ce036ec0e69a11432a870a4ec66ade96,Curve.fi Factory Crypto Pool: APW/ETH,0,0,1657712322,15134061


In [45]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.6030646909652808

In [46]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.906398714717815

Liquidity Pools with > $1mn TVL

In [32]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

83

Monthly Active Users

In [33]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [36]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [37]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [38]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [39]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [40]:
np.mean(monthactiveusers) #can chart this easy

27812.85714285714

Implied slippage